In [3]:
from collections import deque
import pickle

def bfs(tree, start):
    visited = set([start])
    queue = deque([(start, 0)])
    distances = {}
    
    while queue:
        node, distance = queue.popleft()
        distances[node] = distance
        
        for neighbor in tree[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, distance + 1))
    
    return distances

def calculate_m_values(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        for h in range(len(tree)):
            if len([x for x in distances.values() if x <= h]) >= k:
                m_values.append(h)
                break
    
    return m_values

def greedy_labeling(tree, k, m_values):
    nodes = sorted(range(len(tree)), key=lambda x: m_values[x])
    labels = [None] * len(tree)
    
    for node in nodes:
        if labels[node] is None:
            available_labels = set(range(k))
            distances = bfs(tree, node)
            neighbors = [x for x in distances if distances[x] <= m_values[node]]

            for neighbor in neighbors:
                if labels[neighbor] in available_labels:
                    available_labels.remove(labels[neighbor])
                if not available_labels:
                    break

            if available_labels:
                neighborhood_label_frequencies = {label: 0 for label in range(k)}
                for neighbor in neighbors:
                    if labels[neighbor] is not None:
                        neighborhood_label_frequencies[labels[neighbor]] += 1
                
                least_frequent_label = min(available_labels, key=lambda label: neighborhood_label_frequencies[label])
                labels[node] = least_frequent_label

            else:
                # Find the least used label in the neighborhood
                idx_list = range(k)
                label_counts = {label: 0 for label in idx_list}
                for neighbor in neighbors:
                    if labels[neighbor] is not None:
                        label_counts[labels[neighbor]] += 1
                least_used_label = min(label_counts, key=label_counts.get)      
                labels[node] = least_used_label

            # Check if all k labels are distributed in the m-hop neighborhood
            neighborhood_labels = set(labels[neighbor] for neighbor in neighbors if labels[neighbor] is not None)
            missing_labels = set(range(k)) - neighborhood_labels
            
            found = False
            h = 1
            for missing_label in missing_labels:
                while True:
                    # Find an unlabeled node in the neighborhood and assign it the missing label
                    extended_neighbors = [x for x in distances if distances[x] == h]
                    if extended_neighbors == [] and [x for x in distances if distances[x] == h+1] == [] and [x for x in distances if distances[x] == h+2] == []:
                        break
                    for neighbor in extended_neighbors:
                            if labels[neighbor] is None:
                                labels[neighbor] = missing_label
                                found = True
                                break
                    if found:
                        break
                    else:
                        h += 1

    return labels

def load_input(file_adjlist, file_k_values):
    with open(file_adjlist, 'rb') as f:
        adj_list_of_trees = pickle.load(f)
    with open(file_k_values, 'rb') as f:
        k_values = pickle.load(f)
    
    return zip(adj_list_of_trees, k_values)    

def save_output(file_name, output_list):
    with open(file_name, 'wb') as f:
        pickle.dump(output_list, f)

def main():
    # Load input instances
    data_input = load_input('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values')
    # data_input = load_input('Small_Examples_of_AdjLists_of_Trees', 'Small_Examples_of_k_values')
    # data_input = load_input('Medium_Examples_of_AdjLists_of_Trees', 'Medium_Examples_of_k_values')
    # data_input = load_input('Large_Examples_of_AdjLists_of_Trees', 'Large_Examples_of_k_values')

    # Run algorithm and save results
    
    results = []
    for tree, k in data_input:
        m_values = calculate_m_values(tree, k)
        labels = greedy_labeling(tree, k, m_values)
        results.append(labels)

    save_output('example_solutions', results)

if __name__ == '__main__':
    main()

In [4]:
def bfs(tree, start):
    visited = set()
    queue = [(start, 0)]
    distances = {}

    while queue:
        current, depth = queue.pop(0)
        visited.add(current)
        distances[current] = depth
        for neighbor in tree[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth + 1))

    return distances

def r(tree, k, solution):
    r_values = []
    for node, label in enumerate(solution):
        label_set = set()
        distances = bfs(tree, node)
        h = 0
        while len(label_set) < k:
            label_set.update({solution[i] for i, dist in distances.items() if dist <= h})
            h += 1
        r_values.append(h - 1)
    return r_values

def m(tree, k):
    m_values = []
    for node in range(len(tree)):
        distances = bfs(tree, node)
        h = 0
        while len([1 for i, dist in distances.items() if dist <= h]) < k:
            h += 1
        m_values.append(h)
    return m_values

def compute_proximity_ratios(file_adjlist, file_k_values, file_solutions):
    input_instances = load_input(file_adjlist, file_k_values)
    solutions = load_solutions(file_solutions)

    proximity_ratios = []
    for (tree, k), solution in zip(input_instances, solutions):
        r_values = r(tree, k, solution)
        m_values = m(tree, k)

        ratios = [rv / mv for rv, mv in zip(r_values, m_values)]
        max_ratio = max(ratios)
        proximity_ratios.append(max_ratio)

    return proximity_ratios

def load_solutions(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def main():
    # Compute proximity ratios for small solutions
    small_proximity_ratios = compute_proximity_ratios('Examples_of_AdjLists_of_Trees', 'Examples_of_k_values', 'example_solutions')
    print("Proximity ratios:", small_proximity_ratios)

if __name__ == '__main__':
    main()
    sols = load_solutions('example_solutions')
    for sol in sols:
        print(sol)

Proximity ratios: [1.0, 1.0, 1.5, 1.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0, 1, 2, 2, 3, 4, 5, 3, 4, 5, 4, 5]
[0, 1, 2, 2, 0, 0, 2, 1, 2, 0, 1, 2]
[1, 0, 3, 2, 4, 5, 6, 1, 0, 0, 3, 4]
[1, 0, 3, 4, 2, 6, 1, 0, 2, 3, 4, 5]
[1, 0, 2, 3, 0, 0, 1, 2, 0, 2, 1, 3]
[5, 6, 4, 1, 0, 2, 3, 5, 6, 0, 1]
[0, 1, 2, 3, 6, 7, 4, 5, 8, 9, 7, 9]
[1, 0, 3, 5, 2, 4, 6, 4, 6, 7, 7, 0]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1]
[4, 1, 0, 8, 5, 2, 3, 6, 7, 0, 1, 2]
